Initialize environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m pip install pyyaml==5.1

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
!pip install funcy

In [ ]:
!pip install pyrealsense2

In [ ]:
!python -c "import skimage; print(skimage.__version__)"

In [ ]:
# Update pip
!python -m pip install -U pip
# Install scikit-image
!python -m pip install -U scikit-image

In [ ]:
!python -m pip install -U scikit-image[optional]

In [ ]:
# Grab the latest source
!git checkout main
!git pull upstream main
# Update the installation
!pip install -e . -vv

In [ ]:
%matplotlib inline

In [ ]:
# some basic setup:
# setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
# import pyrealsense2 as rs # Intel RealSense cross-platform open-source API
import matplotlib.pyplot as plt   
import itertools as it           
import os, json, cv2, math, random, argparse, funcy, torch, torchvision
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.summary.summary_iterator import summary_iterator
from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg, CfgNode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, CityscapesInstanceEvaluator

print("Environment Ready")

Split coco annotations (json) into train, validation and test sets - **just for the first time !**

In [ ]:
# def save_coco(file, info, licenses, images, annotations, categories):
#     with open(file, 'w') as coco:
#         json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
#             'annotations': annotations, 'categories': categories}, coco, indent=2, sort_keys=True)

# def filter_annotations(annotations, images):
#     image_ids = funcy.lmap(lambda i: int(i['id']), images)
#     return funcy.lfilter(lambda a: int(a['image_id']) in image_ids, annotations)

# def main():
#   annotations = '/content/drive/MyDrive/Colab Notebooks/tesis/data/instances_default.json'
#   ratio = 0.8
#   train_path = '/content/drive/MyDrive/Colab Notebooks/tesis/data/train_dataset.json'
#   val_path = '/content/drive/MyDrive/Colab Notebooks/tesis/data/val_test_dataset.json'
#   having_annotations = True
#   with open(annotations, 'r', encoding='UTF-8') as annotations_file:
#       coco = json.load(annotations_file)
#       info = coco['info']
#       licenses = coco['licenses']
#       images = coco['images']
#       annotations = coco['annotations']
#       categories = coco['categories']
#       number_of_images = len(images)
#       images_with_annotations = funcy.lmap(lambda a: int(a['image_id']), annotations)
#       if having_annotations:
#           images = funcy.lremove(lambda i: i['id'] not in images_with_annotations, images)
#       x, y = train_test_split(images, train_size=ratio)
#       save_coco(train_path, info, licenses, x, filter_annotations(annotations, x), categories)
#       save_coco(val_path, info, licenses, y, filter_annotations(annotations, y), categories)
#       print('finished')

# main()

In [ ]:
# def save_coco(file, info, licenses, images, annotations, categories):
#     with open(file, 'w') as coco:
#         json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
#             'annotations': annotations, 'categories': categories}, coco, indent=2, sort_keys=True)

# def filter_annotations(annotations, images):
#     image_ids = funcy.lmap(lambda i: int(i['id']), images)
#     return funcy.lfilter(lambda a: int(a['image_id']) in image_ids, annotations)

# def main():
#   annotations = '/content/drive/MyDrive/Colab Notebooks/tesis/data/val_test_dataset.json'
#   ratio = 0.5
#   val_path = '/content/drive/MyDrive/Colab Notebooks/tesis/data/val_dataset.json'
#   test_path = '/content/drive/MyDrive/Colab Notebooks/tesis/data/test_dataset.json'
#   having_annotations = True
#   with open(annotations, 'r', encoding='UTF-8') as annotations_file:
#       coco = json.load(annotations_file)
#       info = coco['info']
#       licenses = coco['licenses']
#       images = coco['images']
#       annotations = coco['annotations']
#       categories = coco['categories']
#       number_of_images = len(images)
#       images_with_annotations = funcy.lmap(lambda a: int(a['image_id']), annotations)
#       if having_annotations:
#           images = funcy.lremove(lambda i: i['id'] not in images_with_annotations, images)
#       x, y = train_test_split(images, train_size=ratio)
#       save_coco(val_path, info, licenses, x, filter_annotations(annotations, x), categories)
#       save_coco(test_path, info, licenses, y, filter_annotations(annotations, y), categories)
#       print('finished')

# main()

Prepare the dataset

In [ ]:
DatasetCatalog.clear()

In [ ]:
register_coco_instances("train_dataset", {}, "/content/drive/MyDrive/Colab Notebooks/tesis/data/train_dataset.json", "/content/drive/MyDrive/Colab Notebooks/tesis/data/")
register_coco_instances("val_dataset", {}, "/content/drive/MyDrive/Colab Notebooks/tesis/data/val_dataset.json", "/content/drive/MyDrive/Colab Notebooks/tesis/data/")
# register_coco_instances("test_dataset", {}, "/content/drive/MyDrive/Colab Notebooks/tesis/all_images/test_dataset.json", "/content/drive/MyDrive/Colab Notebooks/tesis/all_images/")

Basic configuration
https://detectron2.readthedocs.io/en/latest/modules/config.html

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# tha Back Bone is ResNet50

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ("val_dataset",)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
cfg.MODEL.BACKBONE.FREEZE_AT = 2
cfg.SOLVER.BASE_LR = 0.01  
cfg.SOLVER.MAX_ITER = 2080
cfg.SOLVER.IMS_PER_BATCH = 2 # the number of training images per step/iteration
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir output

Evaluation

In [ ]:
# # AP50 Evaluation results for segm (IoU=0.7)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

evaluator = COCOEvaluator("val_dataset", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "val_dataset")
print(inference_on_dataset(predictor.model, val_loader, evaluator))